In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import csv
import pandas as pd
import pickle

max_length = 20

In [2]:
with open('vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)
        
len(vocab)
# vocab.get('->', -1), vocab

634

# Load Embeddings

In [3]:
print("loading embeddings")
#     with open('cooc.pkl', 'rb') as f:
#         cooc = pickle.load(f)
embeddings = np.load('embeddings.npy')

loading embeddings


In [4]:
embeddings.shape

(634, 20)

# Loading training set

In [5]:
#Read Data neg and pos with corresponding label

raw_data_neg = pd.read_csv('train_neg_small.txt', header=None, sep="\n", encoding='utf-8', names=['text'],
                               error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE).drop_duplicates()
# raw_data_neg['text'] = raw_data_neg['text']
raw_data_neg['label'] = 0
# raw_data_neg = raw_data_neg[:5000]

raw_data_pos = pd.read_csv('train_pos_small.txt', header=None, sep="\n", encoding='utf-8', names=['text'],
                               error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE).drop_duplicates()
raw_data_pos['label'] = 1
# raw_data_pos['text'] = raw_data_pos['text']
# raw_data_pos = raw_data_pos[:5000]
    
raw_data_train = pd.concat([raw_data_neg, raw_data_pos], ignore_index=True)


raw_data_test = pd.read_csv('test_data.txt', header=None, sep="\n", encoding='utf-8', names=['text'],
                               error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE).drop_duplicates()


raw_data_test['id'], raw_data_test['text'] = raw_data_test['text'].apply(lambda x: x.split(',')[0]), raw_data_test['text'].apply(lambda x: ','.join(x.split(',')[1:]))


In [8]:
raw_data_test

,text,id,processed_text
0,sea doo pro sea scooter ( sports with the port...,1,"[103, 23, 7, 6, 259, 15, 7, 10, 26, 20]"
1,<user> shucks well i work all week so now i ca...,2,"[0, 76, 2, 193, 45, 399, 27, 67, 2, 109, 102, ..."
2,i cant stay away from bug thats my baby,3,"[2, 611, 259, 69, 335, 11, 250]"
3,<user> no ma'am ! ! ! lol im perfectly fine an...,4,"[0, 83, 1, 1, 1, 42, 113, 362, 10, 68, 478, 241]"
4,"whenever i fall asleep watching the tv , i alw...",5,"[2, 586, 233, 7, 6, 2, 163, 46, 23, 8]"
5,<user> he needs to get rid of that thing ! it ...,6,"[0, 117, 428, 5, 35, 19, 16, 334, 1, 14, 12, 4..."
6,its whatever . in a terrible mood ( (,7,"[140, 328, 4, 15, 8, 103, 103]"
7,"yesss ! rt <user> <user> thanks jordan , i lov...",8,"[1, 41, 0, 0, 36, 6, 2, 22, 3, 10, 34, 158, 31..."
8,my friend <user> text me to check up on me las...,9,"[11, 361, 0, 212, 12, 5, 225, 46, 18, 12, 243,..."
9,<user> #followback please . when will ur #unit...,10,"[0, 59, 4, 52, 50, 144, 102, 5, 10, 9, 9]"


# Construct Features for the Training Texts
convert each word in each tweet to a number (which indicates how frequent it is)

In [9]:
# raw_data.text[0].strip().split()
raw_data_train['processed_text'] = raw_data_train.text.apply(lambda x: x.strip().split()).apply(lambda x: np.array([vocab.get(y, -1) for y in x if vocab.get(y, -1) >= 0]))
raw_data_test['processed_text'] = raw_data_test.text.apply(lambda x: x.strip().split()).apply(lambda x: np.array([vocab.get(y, -1) for y in x if vocab.get(y, -1) >= 0]))

raw_data_train.processed_text
raw_data_test.processed_text

0                 [103, 23, 7, 6, 259, 15, 7, 10, 26, 20]
1       [0, 76, 2, 193, 45, 399, 27, 67, 2, 109, 102, ...
2                         [2, 611, 259, 69, 335, 11, 250]
3        [0, 83, 1, 1, 1, 42, 113, 362, 10, 68, 478, 241]
4                  [2, 586, 233, 7, 6, 2, 163, 46, 23, 8]
5       [0, 117, 428, 5, 35, 19, 16, 334, 1, 14, 12, 4...
6                          [140, 328, 4, 15, 8, 103, 103]
7       [1, 41, 0, 0, 36, 6, 2, 22, 3, 10, 34, 158, 31...
8       [11, 361, 0, 212, 12, 5, 225, 46, 18, 12, 243,...
9               [0, 59, 4, 52, 50, 144, 102, 5, 10, 9, 9]
10                        [210, 126, 19, 288, 35, 46, 75]
11      [23, 0, 3, 14, 1, 1, 1, 206, 456, 22, 1, 1, 88...
12      [0, 17, 68, 31, 37, 23, 8, 302, 39, 6, 566, 72...
13      [0, 28, 5, 56, 14, 15, 4, 37, 44, 81, 28, 187,...
14      [0, 59, 135, 186, 5, 1, 16, 111, 24, 179, 1, 1...
15                                          [97, 122, 67]
16            [420, 110, 25, 576, 103, 30, 17, 2, 26, 20]
17      [196, 

Average word vectors over all words of the tweet

In [11]:
raw_data_train.processed_text[0], raw_data_test.processed_text[0]

(array([103, 631,  19, 631,  88,  10,   7,  30, 282,  26,  20]),
 array([103,  23,   7,   6, 259,  15,   7,  10,  26,  20]))

In [12]:
# raw_data_test[raw_data_test['id']== 1]
raw_data_test.loc[4588]

text              <user> its freezing haha bas helw helw .. ta3a...
id                                                             4589
processed_text           [0, 140, 65, 31, 4, 52, 43, 3, 108, 82, 9]
Name: 4588, dtype: object

In [24]:
 embeddings.mean(0).reshape(1, -1).shape

(1, 20)

In [29]:
features_train = np.zeros((raw_data_train['processed_text'].shape[0], max_length))
average_word_vectors_train = raw_data_train['processed_text'].apply(lambda x: embeddings[x, :]).apply(lambda x: x.mean(0))

for (i, doc) in enumerate(average_word_vectors_train):
    features_train[i, :doc.shape[0]] = doc


X_train, y_train = features_train, raw_data_train.label


features_test = np.zeros((raw_data_test['processed_text'].shape[0], max_length))
# average_word_vectors_test = raw_data_test['processed_text'].apply(lambda x: embeddings[x, :] if len(x) > 0 else np.zeros(max_length)).apply(lambda x: x.mean(0))
average_word_vectors_test = raw_data_test['processed_text'].apply(lambda x: embeddings[x, :] if len(x) > 0 else embeddings.mean(0).reshape(1, -1)).apply(lambda x: x.mean(0))

# for (i, asd) in enumerate(raw_data_test['processed_text']):
#     print("i: ", i)
#     print(embeddings[asd, :], asd.shape)



for (i, doc) in enumerate(average_word_vectors_test):
#     print("i: ", i, doc)
    features_test[i, :doc.shape[0]] = doc

average_word_vectors_test

X_test = features_test

In [30]:
X_train.shape, y_train.shape, embeddings.shape, X_test.shape

((1865, 20), (1865,), (634, 20), (10000, 20))

# Train a Linear Classifier
Let's start with logistic regression

In [31]:
model = LogisticRegression(random_state=0, solver='saga', multi_class='ovr')
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=0, warm_start=False)

# Prediction
Predict labels for all tweets in the test set.

In [34]:
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})


In [35]:
y_test = model.predict(X_test)

In [ ]:
create_csv_submission(raw_data_test['id'], y_test, 'dummy_submission')